<h1> RUN ONLY IN GOOGLE COLAB </h1>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [36]:
from google.colab import files

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.notebook import tqdm 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [3]:
path = os.getcwd()
print(path)
filenames = os.listdir(path)
print(filenames)

/content
['.config', 'gdrive', 'sample_data']


In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
# Unzip the files here 
!unzip -q '/content/gdrive/My Drive/Ethn-app/data.zip'

In [7]:
# Load the csv and append path of the images to it
train_df = pd.read_csv("/content/face_data/train.csv")

def add_path(image_name):
    return '/content/face_data/train_images/' + image_name 

train_df['filename'] = train_df['filename'].apply(add_path)
train_df['race'] = train_df['race'].values.astype(str)
train_df.head()

,filename,age,gender,race
0,/content/face_data/train_images/1_0_2_20161219...,1,0,2
1,/content/face_data/train_images/28_1_3_2017010...,28,1,3
2,/content/face_data/train_images/24_1_3_2016122...,24,1,3
3,/content/face_data/train_images/42_1_3_2017010...,42,1,3
4,/content/face_data/train_images/25_0_3_2017010...,25,0,3


In [8]:
SEED = 37
EPOCHS = 30
IMG_SIZE = 256

In [9]:
datagen = ImageDataGenerator(rescale=1/255.0, validation_split=0.1)

In [10]:
train_generator_gender = datagen.flow_from_dataframe(dataframe=train_df,
                                             x_col='filename',
                                             y_col='gender',
                                             subset='training',
                                             batch_size=128,
                                             seed=SEED,
                                             target_size=(IMG_SIZE, IMG_SIZE),
                                             class_mode='raw'
                                             )


validation_generator_gender = datagen.flow_from_dataframe(dataframe=train_df,
                                             x_col='filename',
                                             y_col='gender',
                                             subset='validation',
                                             batch_size=128,
                                             seed=SEED,
                                             target_size=(IMG_SIZE, IMG_SIZE),
                                             class_mode='raw'
                                             )

# ------------------------------------------------------------------------------
train_generator_race = datagen.flow_from_dataframe(dataframe=train_df,
                                             x_col='filename',
                                             y_col='race',
                                             subset='training',
                                             batch_size=128,
                                             seed=SEED,
                                             target_size=(IMG_SIZE, IMG_SIZE),
                                             class_mode='categorical'
                                             )


validation_generator_race = datagen.flow_from_dataframe(dataframe=train_df,
                                             x_col='filename',
                                             y_col='race',
                                             subset='validation',
                                             batch_size=128,
                                             seed=SEED,
                                             target_size=(IMG_SIZE, IMG_SIZE),
                                             class_mode='categorical'
                                             )


# ------------------------------------------------------------------------------
train_generator_age = datagen.flow_from_dataframe(dataframe=train_df,
                                             x_col='filename',
                                             y_col='age',
                                             subset='training',
                                             batch_size=128,
                                             seed=SEED,
                                             target_size=(IMG_SIZE, IMG_SIZE),
                                             class_mode='raw'
                                             )


validation_generator_age = datagen.flow_from_dataframe(dataframe=train_df,
                                             x_col='filename',
                                             y_col='age',
                                             subset='validation',
                                             batch_size=128,
                                             seed=SEED,
                                             target_size=(IMG_SIZE, IMG_SIZE),
                                             class_mode='raw'
                                             )

Found 8801 validated image filenames.
Found 977 validated image filenames.
Found 8801 validated image filenames belonging to 5 classes.
Found 977 validated image filenames belonging to 5 classes.
Found 8801 validated image filenames.
Found 977 validated image filenames.


In [11]:
def model_arch_gender():
    inp = tf.keras.layers.Input(shape = (IMG_SIZE, IMG_SIZE, 3), name = 'inp')
    x = tf.keras.layers.Conv2D(32, (3,3), strides = (1, 1), padding = 'same')(inp)
    x = tf.keras.layers.BatchNormalization(axis = -1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Conv2D(64, (3,3), strides = (1, 1), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis = -1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Conv2D(128, (3,3), strides = (1, 1), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis = -1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Conv2D(256, (3,3), strides = (1, 1), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis = -1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Flatten()(x)
    
    out = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)
    
    model = tf.keras.models.Model(inp, [out])
    model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer = 'Adam', metrics = ['accuracy'])
    return model

In [12]:
def model_arch_race():
    inp = tf.keras.layers.Input(shape = (IMG_SIZE, IMG_SIZE, 3), name = 'inp')
    x = tf.keras.layers.Conv2D(32, (3,3), strides = (1, 1), padding = 'same')(inp)
    x = tf.keras.layers.BatchNormalization(axis = -1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Conv2D(64, (3,3), strides = (1, 1), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis = -1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Conv2D(128, (3,3), strides = (1, 1), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis = -1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Conv2D(256, (3,3), strides = (1, 1), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis = -1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Flatten()(x)
    
    out = tf.keras.layers.Dense(5, activation = 'softmax')(x)
    
    model = tf.keras.models.Model(inp, [out])
    model.compile(loss = tf.keras.losses.CategoricalCrossentropy(), optimizer = 'Adam', metrics = ['accuracy'])
    return model

In [32]:
def model_arch_age():
    inp = tf.keras.layers.Input(shape = (IMG_SIZE, IMG_SIZE, 3), name = 'inp')
    x = tf.keras.layers.Conv2D(32, (3,3), strides = (1, 1), padding = 'same')(inp)
    x = tf.keras.layers.BatchNormalization(axis = -1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Conv2D(64, (3,3), strides = (1, 1), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis = -1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Conv2D(128, (3,3), strides = (1, 1), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis = -1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Conv2D(256, (3,3), strides = (1, 1), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis = -1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Flatten()(x)
    
    x = tf.keras.layers.Dense(256, activation = 'softmax')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(128, activation = 'softmax')(x)
    out = tf.keras.layers.Dense(1, activation='linear')(x)
    
    model = tf.keras.models.Model(inp, [out])
    model.compile(loss = tf.keras.losses.MeanAbsoluteError(), optimizer = 'Adam', metrics = tf.keras.metrics.MAE)
    return model

In [18]:
train_step_size = train_generator_gender.n // train_generator_gender.batch_size
val_step_size = validation_generator_gender.n // validation_generator_gender.batch_size
print(train_step_size)
print(val_step_size)

68
7


In [19]:
# Define the callbacks
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.4, patience=2, verbose=1, mode='min')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor ='val_loss', mode = 'min', patience = 3, 
                                                      verbose = 1, min_delta = 0.0001, restore_best_weights = True)

mse_save_gender = tf.keras.callbacks.ModelCheckpoint('model_gender.hdf5', monitor='val_loss', verbose=0, save_best_only=True,
                                                     mode='min')

mse_save_age = tf.keras.callbacks.ModelCheckpoint('model_age.hdf5', monitor='val_loss', verbose=0, save_best_only=True,
                                                     mode='min')

mse_save_race = tf.keras.callbacks.ModelCheckpoint('model_race.hdf5', monitor='val_loss', verbose=0, save_best_only=True,
                                                     mode='min')

In [33]:
# Load all the models
gender_model = model_arch_gender()
race_model = model_arch_race()
age_model = model_arch_age()

In [23]:
gender_model.fit_generator(train_generator_gender, steps_per_epoch=train_step_size, epochs = EPOCHS, verbose = 1,
                   validation_data=validation_generator_gender, validation_steps=val_step_size,
                   callbacks=[lr_callback, early_stopping, mse_save_gender])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
68/68 [==============================] - 49s 725ms/step - loss: 2.1916 - accuracy: 0.6450 - val_loss: 0.6707 - val_accuracy: 0.6239 - lr: 0.0010
Epoch 2/30
68/68 [==============================] - 45s 669ms/step - loss: 0.5466 - accuracy: 0.7511 - val_loss: 0.8548 - val_accuracy: 0.4632 - lr: 0.0010
Epoch 3/30
68/68 [==============================] - ETA: 0s - loss: 0.4384 - accuracy: 0.7966
Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0004000000189989805.
68/68 [==============================] - 45s 668ms/step - loss: 0.4384 - accuracy: 0.7966 - val_loss: 0.7131 - val_accuracy: 0.4598 - lr: 0.0010
Epoch 4/30
68/68 [==============================] - 46s 669ms/step - loss: 0.3729 - accuracy: 0.8300 - val_loss: 0.7032 - val_accuracy: 0.5112 - lr: 4.0000e-04
Epoch 00004: early stopping


In [34]:
age_model.fit_generator(train_generator_age, steps_per_epoch=train_step_size, epochs = EPOCHS, verbose = 1,
                   validation_data=validation_generator_age, validation_steps=val_step_size,
                   callbacks=[lr_callback, early_stopping, mse_save_age])

Epoch 1/30
68/68 [==============================] - 46s 676ms/step - loss: 29.4175 - mean_absolute_error: 29.4175 - val_loss: 28.4556 - val_mean_absolute_error: 28.4556 - lr: 0.0010
Epoch 2/30
68/68 [==============================] - 46s 675ms/step - loss: 29.2534 - mean_absolute_error: 29.2534 - val_loss: 28.0001 - val_mean_absolute_error: 28.0001 - lr: 0.0010
Epoch 3/30
68/68 [==============================] - 45s 663ms/step - loss: 29.1367 - mean_absolute_error: 29.1367 - val_loss: 28.0720 - val_mean_absolute_error: 28.0720 - lr: 0.0010
Epoch 4/30
68/68 [==============================] - ETA: 0s - loss: 28.9794 - mean_absolute_error: 28.9794
Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0004000000189989805.
68/68 [==============================] - 45s 662ms/step - loss: 28.9794 - mean_absolute_error: 28.9794 - val_loss: 28.0172 - val_mean_absolute_error: 28.0172 - lr: 0.0010
Epoch 5/30
68/68 [==============================] - 45s 668ms/step - loss: 28.8949 - mean_absolu

In [27]:
race_model.fit_generator(train_generator_race, steps_per_epoch=train_step_size, epochs = EPOCHS, verbose = 1,
                   validation_data=validation_generator_race, validation_steps=val_step_size,
                   callbacks=[lr_callback, early_stopping, mse_save_race])

Epoch 1/30
68/68 [==============================] - 45s 667ms/step - loss: 6.2403 - accuracy: 0.4697 - val_loss: 1.3238 - val_accuracy: 0.5257 - lr: 0.0010
Epoch 2/30
68/68 [==============================] - 45s 665ms/step - loss: 1.1394 - accuracy: 0.6068 - val_loss: 1.3306 - val_accuracy: 0.5301 - lr: 0.0010
Epoch 3/30
68/68 [==============================] - ETA: 0s - loss: 1.0539 - accuracy: 0.6433
Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0004000000189989805.
68/68 [==============================] - 45s 667ms/step - loss: 1.0539 - accuracy: 0.6433 - val_loss: 2.0370 - val_accuracy: 0.5324 - lr: 0.0010
Epoch 4/30
68/68 [==============================] - 45s 662ms/step - loss: 0.7726 - accuracy: 0.7162 - val_loss: 1.4924 - val_accuracy: 0.5234 - lr: 4.0000e-04
Epoch 00004: early stopping


In [37]:
files.download('model_age.hdf5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
files.download('model_race.hdf5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>